Clean of loans over $150k

In [251]:
#import dependencies
import pandas as pd
import numpy as np
from datetime import date, datetime
import os, sys 
import io
import ee


In [252]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [253]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1000)

In [254]:
# !pip install -U -q PyDrive

In [255]:
#mounting the google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [256]:
# command shows file path
# ! ls

In [257]:
from google.colab import files

import csv with data types set, check shape, info


In [346]:
# filepath
path = "/content/drive/My Drive/data/PPP Data 150k plus.csv"




In [347]:
#read the csv, create dataframe
date_cols = ["DateApproved"]
df = pd.read_csv(path, index_col=False, dtype={'Jobs Retained':int, 'NAICSCode': str, 'Zip': str, }, parse_dates=date_cols)



In [348]:
#check
df.head(1)

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
0,a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723,813920,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,295.0,2020-04-14,"National Cooperative Bank, National Association",AK - 00


In [349]:
#shape
df.shape


(661218, 16)

In [350]:
#check types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661218 entries, 0 to 661217
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   LoanRange      661218 non-null  object        
 1   BusinessName   661210 non-null  object        
 2   Address        661201 non-null  object        
 3   City           661203 non-null  object        
 4   State          661218 non-null  object        
 5   Zip            661202 non-null  object        
 6   NAICSCode      654435 non-null  object        
 7   BusinessType   659789 non-null  object        
 8   RaceEthnicity  661218 non-null  object        
 9   Gender         661218 non-null  object        
 10  Veteran        661218 non-null  object        
 11  NonProfit      42462 non-null   object        
 12  JobsRetained   620712 non-null  float64       
 13  DateApproved   661218 non-null  datetime64[ns]
 14  Lender         661218 non-null  object        
 15  

In [351]:
#check for nulls
print(df.isna().sum())

LoanRange             0
BusinessName          8
Address              17
City                 15
State                 0
Zip                  16
NAICSCode          6783
BusinessType       1429
RaceEthnicity         0
Gender                0
Veteran               0
NonProfit        618756
JobsRetained      40506
DateApproved          0
Lender                0
CD                    0
dtype: int64


In [352]:
#replace nan value with zeros
df['JobsRetained'].replace('', np.nan, inplace=True)
df.sample()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
379076,"d $350,000-1 million",UNITED WAY OF THE MIDLANDS,2201 Farnam Street Suite 200,OMAHA,NE,68102,813219,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,NaN,2020-04-09,First National Bank of Omaha,NE - 02


In [353]:
#fill null values in jobs saved
df['JobsRetained'] = df['JobsRetained'].fillna(0)

In [354]:
df['JobsRetained']=df['JobsRetained'].astype(int)
df['JobsRetained'].dtypes

dtype('int64')

In [355]:
print(df.isna().sum())

LoanRange             0
BusinessName          8
Address              17
City                 15
State                 0
Zip                  16
NAICSCode          6783
BusinessType       1429
RaceEthnicity         0
Gender                0
Veteran               0
NonProfit        618756
JobsRetained          0
DateApproved          0
Lender                0
CD                    0
dtype: int64


In [356]:
subset=['BusinessName','NAICSCode','Address', 'City', 'Zip', 'BusinessType']
df.dropna(subset=subset, inplace=True)

In [357]:
#create general category column for naics codes
df['NAICS Category'] = df["NAICSCode"].str[:2]
len(df)


653108

In [358]:
df.sample()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD,NAICS Category
76347,"e $150,000-350,000",DSP GROUP INC,2055 GATEWAY PL,SAN JOSE,CA,95110,541511,Corporation,Unanswered,Unanswered,Unanswered,NaN,10,2020-04-14,Bank Leumi USA,CA - 19,54


In [359]:
# #rename columns to match under 150 loans as appropriate
df.columns=['Loan Range', 'Name', 'Address', 'City', 'State', 'Zip', 'NAICS Code', 'Business Type', 'Race', 'Gender', 'Veteran', 'Non Profit', 'Jobs Saved', 'Date', 'Bank', 'CD',  'NAICS Category']



In [360]:
df.sample()


,Loan Range,Name,Address,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank,CD,NAICS Category
557938,c $1-2 million,SCA PROMOTION INC,3030 LYNDON B JOHNSON FWY SUITE 300,DALLAS,TX,75234,541511,Corporation,Unanswered,Male Owned,Unanswered,NaN,79,2020-04-15,Texas Brand Bank,TX - 24,54


In [361]:
# #dropping CD column because we have address and zipcode for location 
df.drop('CD', inplace=True, axis=1)
df.sample()

,Loan Range,Name,Address,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank,NAICS Category
587257,"e $150,000-350,000",HOUSTON CASING SPECIALTIES LLC,195 SOUTHBELT INDUSTRIAL DR,HOUSTON,TX,77047,332812,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,6,2020-04-14,Frost Bank,33


Joins for NAICS data

In [362]:
# df['NAICS Code'] = df['NAICS Code'] != ''
# len(df)

df=df.dropna(subset=['NAICS Category', ])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 653108 entries, 0 to 661209
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Loan Range      653108 non-null  object        
 1   Name            653108 non-null  object        
 2   Address         653108 non-null  object        
 3   City            653108 non-null  object        
 4   State           653108 non-null  object        
 5   Zip             653108 non-null  object        
 6   NAICS Code      653108 non-null  object        
 7   Business Type   653108 non-null  object        
 8   Race            653108 non-null  object        
 9   Gender          653108 non-null  object        
 10  Veteran         653108 non-null  object        
 11  Non Profit      42074 non-null   object        
 12  Jobs Saved      653108 non-null  int64         
 13  Date            653108 non-null  datetime64[ns]
 14  Bank            653108 non-null  obj

In [363]:
n_path = "/content/drive/My Drive/data/naics-cat.csv"
nf_path = "/content/drive/My Drive/data/2017-NAICS-Descriptions.csv"

In [364]:
# Read column names from file
cols = list(pd.read_csv(nf_path, nrows =1))
print(cols)

['Code', 'Title', 'Description']


In [365]:
nf_df = pd.read_csv( nf_path, index_col=0, dtype={'Code':'string'}, na_values = ['<NA>', 'no information'], )
nf_df.sample()

,Title,Description
Code,,
561311,Employment Placement Agencies,This U.S. industry comprises establishments pr...


In [366]:
# Read column names from file
cols = list(pd.read_csv(n_path, nrows =1))
print(cols)

['id', 'cat', 'description', 'biz_count']


In [367]:
ndf = pd.read_csv( n_path, index_col=0, dtype={'cat':'string'}, na_values = ['<NA>', 'no information'], )
ndf.sample()

,cat,description,biz_count
id,,,
15,53,Real Estate Rental and Leasing,868526


In [368]:
# join n_path and df make merged df
df = pd.merge(left=df, right=nf_df, left_on='NAICS Code', right_on='Code', how='left')
df.sample()

,Loan Range,Name,Address,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank,NAICS Category,Title,Description
641957,"d $350,000-1 million","SPORTS SPECIALISTS OF MILWAUKEE, INC.",N104 W13885 Donges Bay Road,GERMANTOWN,WI,53022,423910,Subchapter S Corporation,Unanswered,Unanswered,Non-Veteran,NaN,33,2020-04-08,First Midwest Bank,42,Sporting and Recreational Goods and Supplies M...,This industry comprises establishments primari...


In [369]:
# join n_path and df make merged df
df = pd.merge(left=df, right=ndf, left_on='NAICS Category', right_on='cat', how='left')
df.sample()

,Loan Range,Name,Address,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank,NAICS Category,Title,Description,cat,description,biz_count
476730,"e $150,000-350,000",TAINTED MONKEY LLC,1040 N High St Apt 7,COLUMBUS,OH,43201,722410,Limited Liability Company(LLC),White,Male Owned,Veteran,NaN,44,2020-04-11,The Huntington National Bank,72,Drinking Places (Alcoholic Beverages),This industry comprises establishments known a...,72,Accommodation and Food Services,907516.0


In [370]:
#drop some columns
cols_to_drop = ['cat', 'biz_count', 'Description']
df.drop(cols_to_drop, axis=1, inplace=True)
df.sample()

,Loan Range,Name,Address,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Non Profit,Jobs Saved,Date,Bank,NAICS Category,Title,description
63259,"e $150,000-350,000","ADAM J. JANETTE, D.D.S., INC.",1010 PEACH ST,SAN LUIS OBISPO,CA,93401,621111,Corporation,Unanswered,Male Owned,Non-Veteran,NaN,16,2020-04-28,Pacific Western Bank,62,Offices of Physicians (except Mental Health Sp...,Health Care and Social Assistance


In [371]:
jcount=df.loc[df['Jobs Saved'] == 0]
len(jcount)



87694

Non Profit Data

In [372]:
df['Non Profit'].value_counts()

Y    42074
Name: Non Profit, dtype: int64

In [373]:
np_filter=(df[df['Non Profit'] == 'Y'])
len(np_filter)

42074

In [374]:
# might be interesting to examine the non profit info from all loans, but there isn't enought data
#to deal with it here; splitting the np data into a separate csv to be joined later with under 150k info
np_filter.to_csv('non_profit_over_150k.csv', index=0)
!cp non_profit_over_150k.csv "drive/My Drive/data"

In [375]:
# drop non profit column. 
df = df.drop('Non Profit', 1)
df.sample()

,Loan Range,Name,Address,City,State,Zip,NAICS Code,Business Type,Race,Gender,Veteran,Jobs Saved,Date,Bank,NAICS Category,Title,description
605993,"d $350,000-1 million",ASSOCIATION FOR RESEARCH AND ENLIGHTENMENT,215 67th Street,VIRGINIA BEACH,VA,23451,813110,Non-Profit Organization,Unanswered,Unanswered,Unanswered,0,2020-04-30,Truist Bank d/b/a Branch Banking & Trust Co,81,Religious Organizations,NaN


NAICS Codes

In [376]:
df['NAICS Code'].value_counts()

722511    33586
621111    20884
541110    14286
238220    13119
441110    12689
          ...  
421120        1
315299        1
421850        1
327111        1
313221        1
Name: NAICS Code, Length: 1167, dtype: int64

In [377]:
df['State'].value_counts()

CA    86168
TX    51603
NY    45471
FL    41704
IL    26950
PA    25835
OH    22490
NJ    21213
MI    19701
GA    18156
MA    18138
NC    16236
VA    16111
WA    15708
MN    13675
CO    13247
MD    12834
WI    12307
IN    11738
MO    11413
TN    11237
AZ    11164
OR     9138
LA     8999
CT     8515
AL     7857
SC     7567
OK     6837
UT     6664
KY     6377
IA     5929
KS     5850
NV     5381
AR     4248
NE     4175
MS     3886
NH     3412
ID     3295
HI     3227
NM     2993
ME     2833
DC     2767
RI     2441
WV     2296
MT     2191
ND     2174
DE     2052
SD     1942
PR     1881
AK     1641
VT     1536
WY     1360
GU      296
VI      171
MP       55
AS       19
XX        4
Name: State, dtype: int64

In [378]:
df.shape

(653108, 17)

In [379]:
#check for nulls
print(df.isna().sum())

Loan Range            0
Name                  0
Address               0
City                  0
State                 0
Zip                   0
NAICS Code            0
Business Type         0
Race                  0
Gender                0
Veteran               0
Jobs Saved            0
Date                  0
Bank                  0
NAICS Category        0
Title              5915
description       46066
dtype: int64


In [386]:
df.fillna(value="no info available", inplace=True)

In [388]:
print(df.isna().sum())

Loan Range        0
Name              0
Address           0
City              0
State             0
Zip               0
NAICS Code        0
Business Type     0
Race              0
Gender            0
Veteran           0
Jobs Saved        0
Date              0
Bank              0
NAICS Category    0
Title             0
description       0
dtype: int64


In [389]:
# df['States'] = df['States'].str.lstrip(' ', )

In [390]:
df['State'].nunique()

57

In [391]:
# cleaned file of loans under $150k preserving roughly 4 million rows
df.to_csv('over_150_cleaned.csv', index=0)
!cp over_150_cleaned.csv "drive/My Drive/"